# Downloading the example database and connecting to it

## Packages needed for this project

In [1]:
import urllib.request    # needed for download of the example database
import shutil            # needed for unzipping of the example database

import sqlalchemy as sa
import pandas as pd      # for better table visualisation


### Note on package problems

Currently there exists an incompatibility between:
- the latest version (>=2.0) of `sqlalchemy` package,
- the package `ipython-sql` which provides the `%sql` magic keyword used in the examples below.  

This incompatibility causes the `%sql` magic command to fail.  
A current workaround (which probably will not be necessary in the future) is to install the `jupysql` package (which also downgrades `sqlalchemy` to an older version (1.x.y)).

If you encounter problems with `%sql` try the following (and remember to restart your python kernel):

```python
!pip install jupysql
```

This is how you can check the versions of your packages:

In [2]:
pd.__version__ # Not guaranteed to be fully portable, but works for now.

'2.2.1'

In [3]:
sa.__version__ # Not guaranteed to be fully portable, but works for now.

'2.0.29'

## Downloading of the example database



The SQLite tutorial example database is provided as a `zip` archive in `chinook.zip` file. Download it from the [SQLite tutorial web site](https://www.sqlitetutorial.net/sqlite-sample-database/) (for example using the following Python command):  

In [4]:
urllib.request.urlretrieve("https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip", "chinook.zip")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

*Checkpoint:* The current directory should contain now `chinook.zip` file. Next, unpack the `zip` archive (for example using Python the following Python command):

In [ ]:
shutil.unpack_archive("chinook.zip")

*Checkpoint:* The current directory should contain now `chinook.db` file. This is the example database in SQLite format.

## Connect to the database and execute an SQL query

The following text:

```sql
SELECT * FROM albums LIMIT 5
```

is one of the simplest possible queries in SQL. It requests to provide all (`*`) columns and `5` rows from the database table `albums`. 

In a Python notebook there are several ways to connect to a database and execute the query. These methods differ in type of the object used to return the query result. Study the following examples.

### Using SQLAlchemy engine object

Open the connection to the SQLite database in the `chinook.db` file by creating the database `engine` object as follows:

In [5]:
engine = sa.create_engine("sqlite:///chinook.db")

Using the following code you may send an SQL query `sql` to the database through the `engine`.  
In the `arr` variable you will get the resulting table provided as a list of tuples.

In [6]:
sql = sa.text("SELECT * FROM albums LIMIT 5")
with engine.connect() as conn:
    arr = conn.execute(sql).fetchall()
arr

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3)]

In [10]:
sql = sa.text("SELECT * FROM albums LIMIT 5")
df = pd.read_sql(sql, con=engine)
df

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


### Using Python magic connector

When the Python script works with a single database only and extra Python language extensions are allowed the following notation might be used. The following code creates the database engine connector object in a hidden variable and allows for the magic `%sql` to be used in the code chunks.

In [15]:
%load_ext sql
%sql sqlite:///chinook.db

Using the magic sql connector this is a short way to execute a directly typed single line SQL query and print the result:

In [16]:
%sql SELECT * FROM albums LIMIT 5

 * sqlite:///chinook.db
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


The following code may be used to convert the result to Panda's `DataFrame`:

In [17]:
res = %sql SELECT * FROM albums LIMIT 5
df = res.DataFrame()
df

 * sqlite:///chinook.db
Done.


,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


To print results of longer SQL commands which do not fit in a single line use double-percent notation `%%sql` as below:

In [18]:
%%sql
SELECT trackid, composer, unitprice FROM tracks LIMIT 5

 * sqlite:///chinook.db
Done.


TrackId,Composer,UnitPrice
1,"Angus Young, Malcolm Young, Brian Johnson",0.99
2,None,0.99
3,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",0.99
4,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",0.99
5,Deaffy & R.A. Smith-Diesel,0.99
